In [1]:
import random

import torch
from num2words import num2words

# from transformer_lens import HookedTransformer
from sae_lens import SAE, ActivationsStore, HookedSAETransformer
from tqdm.autonotebook import tqdm
from transformers.utils.logging import disable_progress_bar

from sae_cooccurrence.utils.set_paths import get_git_root

disable_progress_bar()

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
torch.set_grad_enabled(False)

git_root = get_git_root()

if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# from transformer_lens import HookedTransformer

model = HookedSAETransformer.from_pretrained("gemma-2-2b", device=device)

# the cfg dict is returned alongside the SAE since it may contain useful information for analysing the SAE (eg: instantiating an activation store)
# Note that this is not the same as the SAEs config dict, rather it is whatever was in the HF repo, from which we can extract the SAE config dict
# We also return the feature sparsities which are stored in HF for convenience.
sae, cfg_dict, sparsity = SAE.from_pretrained(
    release="gemma-scope-2b-pt-res-canonical",  # <- Release name
    sae_id="layer_12/width_16k/canonical",  # <- SAE id (not always a hook point!)
    device=device,
)

activation_store = ActivationsStore.from_sae(
    model=model,
    sae=sae,
    streaming=True,
    # fairly conservative parameters here so can use same for larger
    # models without running out of memory.
    store_batch_size_prompts=8,
    train_batch_size_tokens=4096,
    n_batches_in_buffer=4,
    device=device,
)

Loaded pretrained model gemma-2-2b into HookedTransformer


Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/sae_lens/training/activations_store.py:245: UserWarning: Dataset is not tokenized. Pre-tokenizing will improve performance and allows for more control over special tokens. See https://jbloomaus.github.io/SAELens/training_saes/#pretokenizing-datasets for more info.
  warnings.warn(


In [71]:
import random

from num2words import num2words


class TokenQuestionGenerator:
    def __init__(self):
        self.colors = ["black", "white"]
        self.special_cases = {"some": "some", "all": "none", "none": "all"}
        self.sections = ["basic", "numeric", "special", "zero"]

    def _number_to_words(self, n: int) -> str:
        """Convert a number to words."""
        return num2words(n)

    def _questions_are_equivalent(self, q1: str, q2: str) -> bool:
        """Compare two questions to check if they are functionally equivalent."""
        # Extract key parts for comparison
        q1_parts = [p.lower() for p in q1.split() if p.lower() not in ["q:", "a:", "and", "are", "of", "them", "my", "tokens", "have", "how", "many"]]
        q2_parts = [p.lower() for p in q2.split() if p.lower() not in ["q:", "a:", "and", "are", "of", "them", "my", "tokens", "have", "how", "many"]]
        return q1_parts == q2_parts

    def generate_zero_case_question(self) -> tuple[str, str]:
        """Generate a question where one color has zero tokens."""
        n_total = random.randint(2, 10)
        colors = random.sample(self.colors, 2)
        held_color, test_color = colors

        question = (
            f"Q: I have {self._number_to_words(n_total)} tokens, and "
            f"all of them are {held_color}. "
            f"How many of my tokens are {test_color}?"
        )
        answer = f"A: zero of them are {test_color}"
        return question, answer

    def generate_numeric_question(self, force_complementary: bool = False) -> tuple[str, str]:
        """Generate a question with numeric values."""
        n_total = random.randint(3, 10)  # Minimum 3 tokens for more interesting cases
        if force_complementary:
            # Generate numbers that sum to interesting complements
            n_color_tokens = random.randint(1, n_total - 1)  # Ensure at least 1 token of each color
        else:
            n_color_tokens = random.randint(0, n_total)
        
        colors = random.sample(self.colors, 2)
        held_color, test_color = colors

        question = (
            f"Q: I have {self._number_to_words(n_total)} tokens, and "
            f"{self._number_to_words(n_color_tokens)} of them are {held_color}. "
            f"How many of my tokens are {test_color}?"
        )
        answer = f"A: {self._number_to_words(n_total - n_color_tokens)} of them are {test_color}"
        return question, answer

    def generate_special_case_question(self) -> tuple[str, str]:
        """Generate a question with special quantifiers (some, all, none)."""
        special_type = random.choice(list(self.special_cases.keys()))
        colors = random.sample(self.colors, 2)
        held_color, test_color = colors
        n_total = random.randint(3, 10)

        question = (
            f"Q: I have {self._number_to_words(n_total)} tokens, and "
            f"{special_type} of them are {held_color}. "
            f"How many of my tokens are {test_color}?"
        )
        answer = f"A: {self.special_cases[special_type]} of them are {test_color}"
        return question, answer

    def generate_test_question(self, force_numeric: bool | None = None, force_special: bool | None = None) -> tuple[str, str]:
        """Generate a test question."""
        if force_numeric and force_special:
            raise ValueError("Cannot force both numeric and special case")

        if force_numeric:
            return self.generate_numeric_question(force_complementary=True)
        elif force_special:
            return self.generate_special_case_question()
        else:
            generators = [
                self.generate_numeric_question,
                self.generate_special_case_question,
                self.generate_zero_case_question
            ]
            return random.choice(generators)()

    def generate_training_set(self, section_counts: dict) -> list[tuple[str, str, str]]:
        """Generate a structured training set with sections."""
        training_set = []
        
        # Basic cases (all/none)
        for _ in range(section_counts.get("basic", 0)):
            q, a = self.generate_special_case_question()
            training_set.append(("# Basic cases - all/none", q, a))
            
        # Numeric cases with specific complementary numbers
        for _ in range(section_counts.get("numeric", 0)):
            q, a = self.generate_numeric_question(force_complementary=True)
            training_set.append(("# Cases with specific numbers", q, a))
            
        # Special cases (some)
        for _ in range(section_counts.get("special", 0)):
            q, a = self.generate_special_case_question()
            training_set.append(("# Cases with 'some'", q, a))
            
        # Zero cases
        for _ in range(section_counts.get("zero", 0)):
            q, a = self.generate_zero_case_question()
            training_set.append(("# Zero cases", q, a))
            
        return training_set

def generate_training_and_test(
    section_counts: dict = {
        "basic": 2,    # all/none cases
        "numeric": 3,  # specific number cases
        "special": 2,  # 'some' cases
        "zero": 2      # explicit zero cases
    },
    force_test_type: str | None = None,
    max_attempts: int = 100
) -> dict:
    """Generate a structured training set and test question."""
    generator = TokenQuestionGenerator()
    
    # Generate test question
    force_numeric = True if force_test_type == "numeric" else None
    force_special = True if force_test_type == "special" else None
    test_question, test_answer = generator.generate_test_question(
        force_numeric=force_numeric, force_special=force_special
    )
    
    # Generate training set with sections
    training_set = []
    attempts = 0
    
    while len(training_set) < sum(section_counts.values()) and attempts < max_attempts:
        current_set = generator.generate_training_set(section_counts)
        
        # Filter out any questions that match the test question
        filtered_set = [
            (section, q, a)
            for section, q, a in current_set
            if not generator._questions_are_equivalent(q, test_question)
        ]
        
        if len(filtered_set) == sum(section_counts.values()):
            training_set = filtered_set
            break
            
        attempts += 1
    
    if attempts >= max_attempts:
        raise RuntimeError("Failed to generate unique training set after maximum attempts")
    
    # Format the introduction with explicit instructions
    introduction = """
Tokens can be either black or white. Complete the following sentences using number words (one, two, three, etc.) never digits (1, 2, 3, etc.).
Remember:
- For 0, use 'zero' not '0'
- The total number of tokens equals the sum of black and white tokens
- Always write numbers as words (e.g., 'two' not '2')

For example, this is correct: 
Q: I have ten tokens, and five of them are black. How many of my tokens are white?
A: five of them are white

Whereas this is incorrect:
Q: I have ten tokens, and five of them are black. How many of my tokens are white?
A: 5 of them are white
"""
    
    return {
        "introduction": introduction.strip(),
        "training_questions": training_set,
        "test_question": test_question,
        "test_answer": test_answer
    }

# Example usage
if __name__ == "__main__":
    # Custom section counts
    section_counts = {
        "basic": 5,    # all/none cases
        "numeric": 3,  # specific number cases
        "special": 2,  # 'some' cases
        "zero": 2      # explicit zero cases
    }
    
    result = generate_training_and_test(
        section_counts=section_counts,
        force_test_type="numeric"  # Can be 'numeric', 'special', or None
    )
    
    print(result["introduction"])
    print("\nTraining Questions:")
    for section, q, a in result["training_questions"]:
        if section:
            print(f"\n{section}")
        print(f"{q}\n{a}\n")
    print("\nTest Question:")
    print(result["test_question"])
    print("\nExpected Answer:")
    print(result["test_answer"])

Tokens can be either black or white. Complete the following sentences using number words (one, two, three, etc.) never digits (1, 2, 3, etc.).
Remember:
- For 0, use 'zero' not '0'
- The total number of tokens equals the sum of black and white tokens
- Always write numbers as words (e.g., 'two' not '2')

For example, this is correct: 
Q: I have ten tokens, and five of them are black. How many of my tokens are white?
A: five of them are white

Whereas this is incorrect:
Q: I have ten tokens, and five of them are black. How many of my tokens are white?
A: 5 of them are white

Training Questions:

# Basic cases - all/none
Q: I have seven tokens, and none of them are black. How many of my tokens are white?
A: all of them are white


# Basic cases - all/none
Q: I have nine tokens, and none of them are white. How many of my tokens are black?
A: all of them are black


# Basic cases - all/none
Q: I have nine tokens, and none of them are white. How many of my tokens are black?
A: all of them a

In [14]:
from functools import partial


def find_max_activation(model, sae, activation_store, feature_idx, num_batches=100):
    """
    Find the maximum activation for a given feature index. This is useful for
    calibrating the right amount of the feature to add.
    """
    max_activation = 0.0

    pbar = tqdm(range(num_batches))
    for _ in pbar:
        tokens = activation_store.get_batch_tokens()

        _, cache = model.run_with_cache(
            tokens,
            stop_at_layer=sae.cfg.hook_layer + 1,
            names_filter=[sae.cfg.hook_name],
        )
        sae_in = cache[sae.cfg.hook_name]
        feature_acts = sae.encode(sae_in).squeeze()

        feature_acts = feature_acts.flatten(0, 1)
        batch_max_activation = feature_acts[:, feature_idx].max().item()
        max_activation = max(max_activation, batch_max_activation)

        pbar.set_description(f"Max activation: {max_activation:.4f}")

    return max_activation


def steering(activations, steering_strength=1.0, steering_vector=None, max_act=1.0):
    # Note if the feature fires anyway, we'd be adding to that here.
    return activations + max_act * steering_strength * steering_vector


def generate_with_steering(
    model,
    sae,
    prompt,
    steering_feature,
    max_act,
    steering_strength=1.0,
    max_new_tokens=95,
):
    input_ids = model.to_tokens(prompt, prepend_bos=sae.cfg.prepend_bos)

    steering_vector = sae.W_dec[steering_feature].to(model.cfg.device)

    steering_hook = partial(
        steering,
        steering_vector=steering_vector,
        steering_strength=steering_strength,
        max_act=max_act,
    )

    # standard transformerlens syntax for a hook context for generation
    with model.hooks(fwd_hooks=[(sae.cfg.hook_name, steering_hook)]):
        output = model.generate(
            input_ids,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            top_p=0.9,
            stop_at_eos=False if device == "mps" else True,
            prepend_bos=sae.cfg.prepend_bos,
        )

    return model.tokenizer.decode(output[0])

In [7]:
def evaluate_model_with_steering(
    model: HookedSAETransformer,
    sae: SAE,
    feature_to_steer: int | None = None,
    steering_strength: float = 1.0,
    max_act: float = 60.0,
    n_numeric: int = 10,
    n_special: int = 10,
    max_new_tokens: int = 5,
) -> dict:
    """
    Test model accuracy on token counting tasks with optional feature steering.

    Args:
        model: The transformer model
        sae: The sparse autoencoder
        feature_to_steer: Feature index to steer, or None for no steering
        steering_strength: Strength of steering (default 1.0)
        max_act: Maximum activation for the steered feature
        n_numeric: Number of numeric test questions
        n_special: Number of special test questions
        max_new_tokens: Maximum tokens to generate for each answer

    Returns:
        Dictionary containing accuracy metrics and test results
    """
    results = {
        "numeric_correct": 0,
        "special_correct": 0,
        "numeric_tests": [],
        "special_tests": [],
    }

    for test_type in ["numeric", "special"]:
        n_tests = n_numeric if test_type == "numeric" else n_special
        for _ in range(n_tests):
            test_data = generate_training_and_test(
                num_training_numeric=40,
                num_training_special=20,
                force_test_type=test_type,
            )

            prompt = (
                test_data["introduction"]
                + "\n\n"
                + "\n\n".join(
                    f"{q}\n{a}\n{label}"
                    for q, a, label in test_data["training_questions"]
                )
                + f"\n\n{test_data['test_question']}\nA: "
            )

            if feature_to_steer is not None:
                generated = generate_with_steering(
                    model,
                    sae,
                    prompt,
                    feature_to_steer,
                    max_act,
                    steering_strength=steering_strength,
                    max_new_tokens=max_new_tokens,
                )
            else:
                # Convert prompt to tokens first
                input_ids = model.to_tokens(prompt, prepend_bos=sae.cfg.prepend_bos)
                output = model.generate(
                    input_ids,
                    max_new_tokens=max_new_tokens,
                    # temperature=0.7,
                    # top_p=0.9,
                    stop_at_eos=False if device == "mps" else True,
                    prepend_bos=sae.cfg.prepend_bos,
                )
                generated = model.tokenizer.decode(output[0])

            test_result = {
                "prompt": prompt,
                "expected": test_data["test_answer"],
                "generated": generated,
                "correct": test_data["test_answer"] in generated,
            }

            if test_type == "numeric":
                results["numeric_tests"].append(test_result)
                if test_result["correct"]:
                    results["numeric_correct"] += 1
            else:
                results["special_tests"].append(test_result)
                if test_result["correct"]:
                    results["special_correct"] += 1

    # Calculate accuracies
    results["numeric_accuracy"] = results["numeric_correct"] / n_numeric
    results["special_accuracy"] = results["special_correct"] / n_special
    results["total_accuracy"] = (
        results["numeric_correct"] + results["special_correct"]
    ) / (n_numeric + n_special)

    return results


# # Example usage:
# if __name__ == "__main__":
#     # Test without steering
#     normal_results = evaluate_model_with_steering(model, sae)
#     print("\nResults without steering:")
#     print(f"Numeric accuracy: {normal_results['numeric_accuracy']:.2%}")
#     print(f"Special accuracy: {normal_results['special_accuracy']:.2%}")
#     print(f"Total accuracy: {normal_results['total_accuracy']:.2%}")

#     print("\nNumeric test results:")
#     for i, test in enumerate(normal_results["numeric_tests"], 1):
#         print(f"\nTest {i}:")
#         question = test["prompt"].split("A: ")[0].splitlines()[-1]
#         print(f"Question: {question}")
#         print(f"Expected: {test['expected']}")
#         print(f"Generated: {test['generated']}")
#         print(f"Correct: {test['correct']}")

#     print("\nSpecial test results:")
#     for i, test in enumerate(normal_results["special_tests"], 1):
#         print(f"\nTest {i}:")
#         question = test["prompt"].split("A: ")[0].splitlines()[-1]
#         print(f"Question: {question}")
#         print(f"Expected: {test['expected']}")
#         print(f"Generated: {test['generated']}")
#         print(f"Correct: {test['correct']}")

#     # Test with feature steering
#     feature_to_steer = 12257  # Replace with your feature of interest
#     steering_strength = 0.0
#     steered_results = evaluate_model_with_steering(
#         model,
#         sae,
#         feature_to_steer=feature_to_steer,
#         steering_strength=steering_strength,
#     )
#     print(
#         f"\nResults with feature {feature_to_steer} steered (strength {steering_strength}):"
#     )
#     print(f"Numeric accuracy: {steered_results['numeric_accuracy']:.2%}")
#     print(f"Special accuracy: {steered_results['special_accuracy']:.2%}")
#     print(f"Total accuracy: {steered_results['total_accuracy']:.2%}")

#     print("\nNumeric test results (with steering):")
#     for i, test in enumerate(steered_results["numeric_tests"], 1):
#         print(f"\nTest {i}:")
#         question = test["prompt"].split("A: ")[0].splitlines()[-1]
#         print(f"Question: {question}")
#         print(f"Expected: {test['expected']}")
#         print(f"Generated: {test['generated']}")
#         print(f"Correct: {test['correct']}")

#     print("\nSpecial test results (with steering):")
#     for i, test in enumerate(steered_results["special_tests"], 1):
#         print(f"\nTest {i}:")
#         question = test["prompt"].split("A: ")[0].splitlines()[-1]
#         print(f"Question: {question}")
#         print(f"Expected: {test['expected']}")
#         print(f"Generated: {test['generated']}")
#         print(f"Correct: {test['correct']}")

#     # Test with feature steering
#     feature_to_steer = 15441  # Replace with your feature of interest
#     steering_strength = 0.0
#     steered_results = evaluate_model_with_steering(
#         model,
#         sae,
#         feature_to_steer=feature_to_steer,
#         steering_strength=steering_strength,
#     )
#     print(
#         f"\nResults with feature {feature_to_steer} steered (strength {steering_strength}):"
#     )
#     print(f"Numeric accuracy: {steered_results['numeric_accuracy']:.2%}")
#     print(f"Special accuracy: {steered_results['special_accuracy']:.2%}")
#     print(f"Total accuracy: {steered_results['total_accuracy']:.2%}")

#     print("\nNumeric test results (with steering):")
#     for i, test in enumerate(steered_results["numeric_tests"], 1):
#         print(f"\nTest {i}:")
#         question = test["prompt"].split("A: ")[0].splitlines()[-1]
#         print(f"Question: {question}")
#         print(f"Expected: {test['expected']}")
#         print(f"Generated: {test['generated']}")
#         print(f"Correct: {test['correct']}")

#     print("\nSpecial test results (with steering):")
#     for i, test in enumerate(steered_results["special_tests"], 1):
#         print(f"\nTest {i}:")
#         question = test["prompt"].split("A: ")[0].splitlines()[-1]
#         print(f"Question: {question}")
#         print(f"Expected: {test['expected']}")
#         print(f"Generated: {test['generated']}")
#         print(f"Correct: {test['correct']}")

In [86]:
def evaluate_model_with_intervention(
    model: HookedSAETransformer,
    sae: SAE,
    feature_ids: int | list[int] | None = None,
    intervention_type: str = "none",  # "none", "steering", or "ablation"
    steering_strength: float = 1.0,
    max_act: float = 60.0,
    n_numeric: int = 10,
    n_special: int = 10,
    max_new_tokens: int = 6,
    temperature: float = 0.0,
    top_p: float = 0.9,
    section_counts: dict | None = None,
) -> dict:
    """
    Test model accuracy on token counting tasks with optional feature steering or ablation.
    Lenient scoring now accepts numeric answers for 'all' cases.
    """
    if section_counts is None:
        section_counts = {
            "basic": 2,     # all/none cases
            "numeric": 3,   # specific number cases
            "special": 2,   # 'some' cases
            "zero": 2       # explicit zero cases
        }

    results = {
        "numeric_correct_strict": 0,
        "numeric_correct_lenient": 0,
        "special_correct_strict": 0,
        "special_correct_lenient": 0,
        "numeric_tests": [],
        "special_tests": [],
    }

    def convert_words_to_digits(text: str) -> str:
        """Convert number words to digits in the text."""
        word_to_digit = {
            "zero": "0",
            "none": "0",
            "one": "1",
            "two": "2",
            "three": "3",
            "four": "4",
            "five": "5",
            "six": "6",
            "seven": "7",
            "eight": "8",
            "nine": "9",
            "ten": "10",
        }
        for word, digit in word_to_digit.items():
            text = text.replace(word, digit)
        return text

    def check_answer_lenient(generated: str, expected: str, question: str) -> bool:
        """
        Check if answer is correct under lenient scoring rules.
        
        Args:
            generated: Generated answer text
            expected: Expected answer text
            question: Original question text (needed to extract total token count)
            
        Returns:
            bool: Whether the answer is correct under lenient scoring
        """
        # Convert both to lowercase for comparison
        generated = generated.lower()
        expected = expected.lower()
        
        # First check if it's an exact match after converting words to digits
        generated_digits = convert_words_to_digits(generated)
        expected_digits = convert_words_to_digits(expected)
        
        if expected_digits in generated_digits:
            return True
            
        # Check if this is an "all of them are" case
        if "all of them are" in expected:
            # Extract the total number from the question
            question_words = question.lower().split()
            try:
                # Find the word after "have" in the question
                have_idx = question_words.index("have")
                total_tokens = convert_words_to_digits(question_words[have_idx + 1])
                
                # Check if the generated answer contains this number
                # Allow both "N of them" and just "N"
                return (
                    total_tokens in generated_digits
                    or f"{total_tokens} of them" in generated_digits
                )
            except (ValueError, IndexError):
                return False
                
        return False

    # Convert single feature_id to list for consistency
    if isinstance(feature_ids, int):
        feature_ids = [feature_ids]

    def ablate_feature_hook(feature_activations, hook=None, feature_ids=None):  # noqa: ARG001
        feature_activations[:, :, feature_ids] = 0
        return feature_activations

    for test_type in tqdm(["numeric", "special"], desc="Testing types"):
        n_tests = n_numeric if test_type == "numeric" else n_special
        for _ in tqdm(range(n_tests), desc=f"Testing {test_type}"):
            # Use new structured prompt generator
            test_data = generate_training_and_test(
                section_counts=section_counts,
                force_test_type=test_type,
            )

            # Construct prompt with sections
            prompt_parts = [test_data["introduction"]]
            
            # Group questions by section
            sections = {}
            for section, q, a in test_data["training_questions"]:
                if section not in sections:
                    sections[section] = []
                sections[section].append((q, a))
            
            # Add each section with its header
            for section, questions in sections.items():
                prompt_parts.append(f"\n\n{section}")
                for q, a in questions:
                    prompt_parts.append(f"{q}\n{a}")
            
            # Add test question
            prompt_parts.append(f"\n\n{test_data['test_question']}\nA: ")
            
            prompt = "\n\n".join(prompt_parts)

            # Convert prompt to tokens
            input_ids = model.to_tokens(prompt, prepend_bos=sae.cfg.prepend_bos)

            if intervention_type == "none" or feature_ids is None:
                output = model.generate(
                    input_ids,
                    max_new_tokens=max_new_tokens,
                    temperature=temperature,
                    top_p=top_p,
                    stop_at_eos=False if device == "mps" else True,
                    prepend_bos=sae.cfg.prepend_bos,
                )

            elif intervention_type == "steering":
                steering_vector = sae.W_dec[feature_ids[0]].to(model.cfg.device)
                steering_hook = partial(
                    steering,
                    steering_vector=steering_vector,
                    steering_strength=steering_strength,
                    max_act=max_act,
                )

                with model.hooks(fwd_hooks=[(sae.cfg.hook_name, steering_hook)]):
                    output = model.generate(
                        input_ids,
                        max_new_tokens=max_new_tokens,
                        temperature=temperature,
                        top_p=top_p,
                        stop_at_eos=False if device == "mps" else True,
                        prepend_bos=sae.cfg.prepend_bos,
                    )

            elif intervention_type == "ablation":
                ablation_hook = partial(ablate_feature_hook, feature_ids=feature_ids)
                model.add_sae(sae)
                hook_point = sae.cfg.hook_name + ".hook_sae_acts_post"

                with model.hooks(fwd_hooks=[(hook_point, ablation_hook)]):
                    output = model.generate(
                        input_ids,
                        max_new_tokens=max_new_tokens,
                        top_p=top_p,
                        temperature=temperature,
                        stop_at_eos=False if device == "mps" else True,
                        prepend_bos=sae.cfg.prepend_bos,
                    )

                model.reset_hooks()
                model.reset_saes()

            generated = model.tokenizer.decode(output[0])  # type: ignore
            generated_answer = generated.split("A: ")[-1].strip()
            expected_answer = test_data["test_answer"].replace("A: ", "").strip()
            test_question = test_data["test_question"].split("\n")[0].strip()

            test_result = {
                "prompt": prompt,
                "expected": expected_answer,
                "generated": generated_answer,
                "correct_strict": expected_answer in generated_answer,
                "correct_lenient": check_answer_lenient(
                    generated_answer, expected_answer, test_question
                ),
                "section_counts": section_counts,
            }

            if test_type == "numeric":
                results["numeric_tests"].append(test_result)
                if test_result["correct_strict"]:
                    results["numeric_correct_strict"] += 1
                if test_result["correct_lenient"]:
                    results["numeric_correct_lenient"] += 1
            else:
                results["special_tests"].append(test_result)
                if test_result["correct_strict"]:
                    results["special_correct_strict"] += 1
                if test_result["correct_lenient"]:
                    results["special_correct_lenient"] += 1

    # Calculate accuracies
    results["numeric_accuracy_strict"] = results["numeric_correct_strict"] / n_numeric
    results["numeric_accuracy_lenient"] = results["numeric_correct_lenient"] / n_numeric
    results["special_accuracy_strict"] = results["special_correct_strict"] / n_special
    results["special_accuracy_lenient"] = results["special_correct_lenient"] / n_special
    results["total_accuracy_strict"] = (
        results["numeric_correct_strict"] + results["special_correct_strict"]
    ) / (n_numeric + n_special)
    results["total_accuracy_lenient"] = (
        results["numeric_correct_lenient"] + results["special_correct_lenient"]
    ) / (n_numeric + n_special)

    # Add section configuration to results
    results["section_counts"] = section_counts

    return results

In [64]:
normal_results = evaluate_model_with_intervention(
    model, sae, intervention_type="none", n_numeric=1, n_special=1, section_counts= {"basic": 20, "numeric": 0, "special": 20, "zero": 0}
)

Testing types:   0%|          | 0/2 [00:00<?, ?it/s]

Testing numeric:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

Testing special:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

In [65]:
normal_results["numeric_tests"]

[{'prompt': "Tokens can be either black or white. Complete the following sentences using number words (one, two, three, etc.) never digits (1, 2, 3, etc.).\nRemember:\n- For 0, use 'zero' not '0'\n- The total number of tokens equals the sum of black and white tokens\n- Always write numbers as words (e.g., 'two' not '2')\n\nFor example, this is correct: \nQ: I have 10 tokens, and 5 of them are black. How many of my tokens are white?\nA: five of them are white\n\nWhereas this is incorrect:\nQ: I have 10 tokens, and 5 of them are black. How many of my tokens are white?\nA: 5 of them are white\n\n\n\n# Basic cases - all/none\n\nQ: I have six tokens, and some of them are white. How many of my tokens are black?\nA: some of them are black\n\nQ: I have four tokens, and some of them are white. How many of my tokens are black?\nA: some of them are black\n\nQ: I have five tokens, and all of them are white. How many of my tokens are black?\nA: none of them are black\n\nQ: I have seven tokens, and 

In [88]:
steering_results_hub = evaluate_model_with_intervention(
    model,
    sae,
    feature_ids=[12257],  # Can ablate multiple features
    intervention_type="steering",
    steering_strength=1.0,
    n_numeric=1,
    n_special=1,
)

Testing types:   0%|          | 0/2 [00:00<?, ?it/s]

Testing numeric:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

TypeError: steering() got an unexpected keyword argument 'hook'

In [ ]:
neg_steering_results_hub = evaluate_model_with_intervention(
    model,
    sae,
    feature_ids=[12257],  # Can ablate multiple features
    intervention_type="ablation",
    n_numeric_test=1,
    n_special_test=1,
)

In [23]:
ablated_results_hub['special_tests']


[{'prompt': "Tokens can be either black or white. Complete the following sentences using number words (one, two, three, etc.) never digits (1, 2, 3, etc.).\nRemember:\n- For 0, use 'zero' not '0'\n- The total number of tokens equals the sum of black and white tokens\n- Always write numbers as words (e.g., 'two' not '2')\n\n\n\n# Basic cases - all/none\n\nQ: I have nine tokens, and all of them are black. How many of my tokens are white?\nA: zero of them are white\n\nQ: I have six tokens, and none of them are white. How many of my tokens are black?\nA: all of them are black\n\n\n\n# Cases with specific numbers\n\nQ: I have eight tokens, and one of them are black. How many of my tokens are white?\nA: seven of them are white\n\nQ: I have three tokens, and one of them are black. How many of my tokens are white?\nA: two of them are white\n\nQ: I have nine tokens, and two of them are black. How many of my tokens are white?\nA: seven of them are white\n\n\n\n# Cases with 'some'\n\nQ: I have ni

In [76]:
# Test without intervention
normal_results = evaluate_model_with_intervention(model, sae, intervention_type="none", section_counts= {"basic": 20, "numeric": 20, "special": 20, "zero": 0}, n_numeric=50, n_special=50)

# # Test with steering
# steered_results = evaluate_model_with_intervention(
#     model,
#     sae,
#     feature_ids=12257,
#     intervention_type="steering",
#     steering_strength=1.0,
# )

Testing types:   0%|          | 0/2 [00:00<?, ?it/s]

Testing numeric:   0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

In [74]:
print(normal_results["numeric_accuracy_strict"])
print(normal_results["numeric_accuracy_lenient"])
print(normal_results["special_accuracy_strict"])
print(normal_results["special_accuracy_lenient"])
# Test with ablation

0.0
1.0
0.0
0.4


In [33]:
normal_results['numeric_tests']

[{'prompt': "Tokens can be either black or white. Complete the following sentences using number words (one, two, three, etc.) never digits (1, 2, 3, etc.).\nRemember:\n- For 0, use 'zero' not '0'\n- The total number of tokens equals the sum of black and white tokens\n- Always write numbers as words (e.g., 'two' not '2')\n\n\n\n# Basic cases - all/none\n\nQ: I have six tokens, and all of them are black. How many of my tokens are white?\nA: none of them are white\n\nQ: I have eight tokens, and some of them are black. How many of my tokens are white?\nA: some of them are white\n\nQ: I have three tokens, and all of them are white. How many of my tokens are black?\nA: none of them are black\n\n\n\n# Cases with specific numbers\n\nQ: I have ten tokens, and four of them are black. How many of my tokens are white?\nA: six of them are white\n\nQ: I have seven tokens, and four of them are white. How many of my tokens are black?\nA: three of them are black\n\nQ: I have six tokens, and two of them

In [75]:
normal_results['special_tests']

[{'prompt': "Tokens can be either black or white. Complete the following sentences using number words (one, two, three, etc.) never digits (1, 2, 3, etc.).\nRemember:\n- For 0, use 'zero' not '0'\n- The total number of tokens equals the sum of black and white tokens\n- Always write numbers as words (e.g., 'two' not '2')\n\nFor example, this is correct: \nQ: I have ten tokens, and five of them are black. How many of my tokens are white?\nA: five of them are white\n\nWhereas this is incorrect:\nQ: I have ten tokens, and five of them are black. How many of my tokens are white?\nA: 5 of them are white\n\n\n\n# Basic cases - all/none\n\nQ: I have seven tokens, and none of them are white. How many of my tokens are black?\nA: all of them are black\n\nQ: I have five tokens, and some of them are white. How many of my tokens are black?\nA: some of them are black\n\nQ: I have five tokens, and all of them are white. How many of my tokens are black?\nA: none of them are black\n\nQ: I have four toke

In [77]:
# Test with ablation
ablated_results_hub = evaluate_model_with_intervention(
    model,
    sae,
    feature_ids=[12257],  # Can ablate multiple features
    intervention_type="ablation",
    n_numeric=50,
    n_special=50,
    section_counts= {"basic": 20, "numeric": 0, "special": 20, "zero": 0}
)

ablated_results_some = evaluate_model_with_intervention(
    model,
    sae,
    feature_ids=[15441],  # Can ablate multiple features
    intervention_type="ablation",
    n_numeric=50,
    n_special=50,
    section_counts= {"basic": 20, "numeric": 20, "special": 0, "zero": 0}
)

ablated_results_all = evaluate_model_with_intervention(
    model,
    sae,
    feature_ids=[12649],  # Can ablate multiple features
    intervention_type="ablation",
    n_numeric=50,
    n_special=50,
    section_counts= {"basic": 20, "numeric": 20, "special": 0, "zero": 0}
)

ablation_results_spokes = evaluate_model_with_intervention(
    model,
    sae,
    feature_ids=[12649, 15441],  # Can ablate multiple features
    intervention_type="ablation",
    n_numeric=50,
    n_special=50,
    section_counts= {"basic": 20, "numeric": 20, "special": 0, "zero": 0}
)

ablation_results_hub_spoke_some = evaluate_model_with_intervention(
    model,
    sae,
    feature_ids=[12257, 15441],  # Can ablate multiple features
    intervention_type="ablation",
    n_numeric=50,
    n_special=50,
    section_counts= {"basic": 20, "numeric": 20, "special": 0, "zero": 0}
)


Testing types:   0%|          | 0/2 [00:00<?, ?it/s]

Testing numeric:   0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

In [78]:

ablated_results_hub_spoke_all = evaluate_model_with_intervention(
    model,
    sae,
    feature_ids=[12257, 12649],  # Can ablate multiple features
    intervention_type="ablation",
    n_numeric=50,
    n_special=50,
    section_counts= {"basic": 20, "numeric": 20, "special": 0, "zero": 0}
)

Testing types:   0%|          | 0/2 [00:00<?, ?it/s]

Testing numeric:   0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

Compare the effect of ablation of the hub alone both spokes alone hub and both spokes together and both spokes together I hope hub and spokes will be more effective than any of these 

In [85]:
print(normal_results["numeric_accuracy_strict"])
print(normal_results["numeric_accuracy_lenient"])
print(normal_results["special_accuracy_strict"])
print(normal_results["special_accuracy_lenient"])


0.0
1.0
0.0
0.62


In [79]:
print(ablated_results_hub["numeric_accuracy_strict"])
print(ablated_results_hub["numeric_accuracy_lenient"])
print(ablated_results_hub["special_accuracy_strict"])
print(ablated_results_hub["special_accuracy_lenient"])

0.0
0.74
0.0
0.6


In [80]:
print(ablated_results_some["numeric_accuracy_strict"])
print(ablated_results_some["numeric_accuracy_lenient"])
print(ablated_results_some["special_accuracy_strict"])
print(ablated_results_some["special_accuracy_lenient"])

0.0
0.84
0.0
0.64


In [81]:
print(ablated_results_all["numeric_accuracy_strict"])
print(ablated_results_all["numeric_accuracy_lenient"])
print(ablated_results_all["special_accuracy_strict"])
print(ablated_results_all["special_accuracy_lenient"])

0.0
0.9
0.0
0.64


In [82]:
print(ablation_results_spokes["numeric_accuracy_strict"])
print(ablation_results_spokes["numeric_accuracy_lenient"])
print(ablation_results_spokes["special_accuracy_strict"])
print(ablation_results_spokes["special_accuracy_lenient"])


0.0
0.88
0.0
0.6


In [83]:
print(ablation_results_hub_spoke_some["numeric_accuracy_strict"])
print(ablation_results_hub_spoke_some["numeric_accuracy_lenient"])
print(ablation_results_hub_spoke_some["special_accuracy_strict"])
print(ablation_results_hub_spoke_some["special_accuracy_lenient"])


0.0
0.86
0.0
0.68


In [84]:
print(ablated_results_hub_spoke_all["numeric_accuracy_strict"])
print(ablated_results_hub_spoke_all["numeric_accuracy_lenient"])
print(ablated_results_hub_spoke_all["special_accuracy_strict"])
print(ablated_results_hub_spoke_all["special_accuracy_lenient"])


0.0
0.94
0.0
0.6
